In [1]:
print("Hello, World!")

Hello, World!


In [ ]:
# 필수 패키지 설치
# !pip install olefile

In [ ]:
import olefile
import zlib

def list_streams(hwp_path):
    """OLE 파일의 스트림과 스토리지를 나열"""
    ole = olefile.OleFileIO(hwp_path)
    print("=== Streams and Storages ===")
    for entry in ole.listdir():
        print("/".join(entry))
    ole.close()

def read_section0(hwp_path):
    """BodyText/Section0 스트림 읽고 zlib 해제"""
    ole = olefile.OleFileIO(hwp_path)
    
    # BodyText/Section0 stream 경로
    stream_path = ["BodyText", "Section0"]

    if ole.exists(stream_path):
        data = ole.openstream(stream_path).read()
        # BodyText는 zlib 압축된 상태
        # HWP 문서 스펙: 첫 4바이트는 레코드 헤더로 간주 가능 (간단화)
        header = data[:4]
        compressed = data[4:]
        try:
            decompressed = zlib.decompress(compressed)
            print("=== Decompressed BodyText (first 200 bytes) ===")
            print(decompressed[:200])
        except Exception as e:
            print("[!] zlib 해제 실패:", e)
    else:
        print("[!] BodyText/Section0 스트림이 없습니다.")

    ole.close()

# if __name__ == "__main__":
#     hwp_file = "sample.hwp"

#     # 1) 스트림 리스트 확인
#     list_streams(hwp_file)

#     # 2) Section0 읽기 및 zlib 해제
#     read_section0(hwp_file)

hwp_file = "sample_file.hwp"

# 1) 스트림 리스트 확인
list_streams(hwp_file)

# 2) Section0 읽기 및 zlib 해제
read_section0(hwp_file)


=== Streams and Storages ===
HwpSummaryInformation
BinData/BIN0001.tif
BodyText/Section0
DocInfo
DocOptions/_LinkDoc
FileHeader
PrvImage
PrvText
Scripts/DefaultJScript
Scripts/JScriptVersion
[!] zlib 해제 실패: Error -3 while decompressing data: incorrect header check


In [5]:
import olefile
import struct
import zlib

def list_streams(hwp_path):
    ole = olefile.OleFileIO(hwp_path)
    print("=== Streams and Storages ===")
    for entry in ole.listdir():
        print("/".join(entry))
    ole.close()

def check_compression(hwp_path):
    ole = olefile.OleFileIO(hwp_path)
    if ole.exists("FileHeader"):
        header_data = ole.openstream("FileHeader").read()
        compression_flag = header_data[36]
        print(f"압축 플래그 값: {compression_flag}")
        return compression_flag & 1 == 1
    ole.close()
    return False

def read_section0(hwp_path, is_compressed):
    ole = olefile.OleFileIO(hwp_path)
    stream_path = ["BodyText", "Section0"]

    if ole.exists(stream_path):
        data = ole.openstream(stream_path).read()
        compressed_size, uncompressed_size = struct.unpack("<HH", data[:4])
        compressed_data = data[4:]

        print(f"압축 크기: {compressed_size}, 해제 후 크기: {uncompressed_size}")

        if is_compressed:
            try:
                # 핵심 포인트: -15로 raw DEFLATE 해제
                decompressed = zlib.decompress(compressed_data, -15)
                print("=== Decompressed BodyText (first 200 bytes) ===")
                print(decompressed[:200])
            except Exception as e:
                print("[!] zlib 해제 실패:", e)
        else:
            print("=== Uncompressed BodyText (first 200 bytes) ===")
            print(data[4:204])

    else:
        print("[!] BodyText/Section0 스트림이 없습니다.")

    ole.close()

hwp_file = "sample_file.hwp"

print("Hello, World!")
list_streams(hwp_file)

is_compressed = check_compression(hwp_file)

read_section0(hwp_file, is_compressed)



Hello, World!
=== Streams and Storages ===
HwpSummaryInformation
BinData/BIN0001.tif
BodyText/Section0
DocInfo
DocOptions/_LinkDoc
FileHeader
PrvImage
PrvText
Scripts/DefaultJScript
Scripts/JScriptVersion
압축 플래그 값: 1
압축 크기: 22212, 해제 후 크기: 20445
[!] zlib 해제 실패: Error -3 while decompressing data: invalid code lengths set


In [3]:
!pip install pyhwpx

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.7 MB 3.4 MB/s eta 0:00:04
   --- ------------------------------------ 1.0/12.7 MB 2.6 MB/s eta 0:00:05
   ---- ----------------------------------- 1.6/12.7 MB 2.4 MB/s eta 0:00:05
   ------ --------------------------------- 2.1/12.7 MB 2.6 MB/s eta 0:00:05
   -------- -----------

In [2]:
from pyhwpx import Hwp

hwp = Hwp(visible=False)  # 백그라운드에서 작업!(마지막엔 꼭 hwp.quit()을 실행해야 함)
hwp.open("D:\KT\document_parser_hwp\experiment\sample_file.hwp")  # 원본문서를 열어서

# for i in range(hwp.PageCount):
#     hwp.MoveDocBegin()  # 문서 처음에서부터
#     hwp.MoveSelPageDown()  # 한 페이지 전체를 선택하고
#     hwp.MoveSelLeft()  # 한 칸 왼쪽으로 가줘야 2페이지(빈 페이지)가 안 생김
#     hwp.save_block_as(f"{i+1}.hwp")  # 블록 저장
#     hwp.DeletePage()  # 저장한 페이지는 삭제
hwp.save_as(f"{i+1}.hwp")  # 마지막(100번째) 페이지는 PageDown으로 블록이 안 잡히므로 별도 저장
hwp.msgbox('작업이 완료되었습니다.')  # 팝업 띄워주기
hwp.quit()  # 작업 완료되면 종료(창이 보이지 않으므로 꼭 코드로 종료하는 걸 잊지 말아야 함

<>:4: SyntaxWarning: invalid escape sequence '\K'
<>:4: SyntaxWarning: invalid escape sequence '\K'
C:\Users\KT\AppData\Local\Temp\ipykernel_5356\458350842.py:4: SyntaxWarning: invalid escape sequence '\K'
  hwp.open("D:\KT\document_parser_hwp\experiment\sample_file.hwp")  # 원본문서를 열어서
C:\Users\KT\AppData\Local\Temp\ipykernel_5356\458350842.py:4: SyntaxWarning: invalid escape sequence '\K'
  hwp.open("D:\KT\document_parser_hwp\experiment\sample_file.hwp")  # 원본문서를 열어서


ModuleNotFoundError: No module named 'pyhwpx'

In [4]:
!pip install pyhwpx

from pyhwpx import Hwp

hwp = Hwp()  # 보안모듈 자동 등록

# 텍스트 삽입
hwp.insert_text("Hello world!")

# win32com 방식으로도 실행 가능
pset = hwp.HParameterSet.HInsertText
pset.Text = "Hello world!"
hwp.HAction.Execute("InsertText", pset.HSet)

# 문서 저장
hwp.save_as("./helloworld.hwp")

# 한/글 종료
hwp.quit()

com_error: (-2147319779, '라이브러리가 등록되지 않았습니다.', None, None)